In [45]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from dnn_app_utils_v3 import load_data
import torch.nn as nn

train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
print(train_x_orig.shape, train_y.shape, test_x_orig.shape, test_y.shape)

train_x = torch.from_numpy(train_x_orig.reshape(-1, 64 * 64 * 3)).type(torch.float32) / 255.
test_x = torch.from_numpy(test_x_orig.reshape(-1, 64 * 64 * 3)).type(torch.float32) / 255.
train_y = torch.from_numpy(train_y.reshape(-1, 1)).type(torch.float32)
test_y = torch.from_numpy(test_y.reshape(-1, 1)).type(torch.float32)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(209, 64, 64, 3) (1, 209) (50, 64, 64, 3) (1, 50)
torch.Size([209, 12288]) torch.Size([209, 1]) torch.Size([50, 12288]) torch.Size([50, 1])


In [47]:
%config Completer.use_jedi = False

In [33]:
train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

In [28]:
criterion = nn.BCELoss()
learning_rate = 0.0075

In [41]:
n_x = 12288 
n_h1 = 20
n_h2 = 10
n_h3 = 5
n_y = 1
model = nn.Sequential(nn.Linear(n_x, n_h1), nn.ReLU(), nn.Linear(n_h1, n_h2),nn.ReLU(),nn.Linear(n_h2, n_h3), nn.ReLU(), nn.Linear(n_h3, n_y),nn.Sigmoid())
model

Sequential(
  (0): Linear(in_features=12288, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=5, bias=True)
  (5): ReLU()
  (6): Linear(in_features=5, out_features=1, bias=True)
  (7): Sigmoid()
)

In [42]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
n_epoch = 1000
for epoch in range(n_epoch):
    for x, y in train_loader:
        out = model(x)
        loss = criterion(out,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_hat = torch.round(out)
        # print(f"ephoch {epoch+1} cost {loss.item()} accuracy {torch.eq(y_hat, y).sum()*100/y.size()[0]} %")

In [44]:
with torch.no_grad():
    for x_test , y_test in test_loader:
        y_pred = model(x_test)
        y_hat2 = torch.round(y_pred)
        print(f"accuracy {torch.eq(y_hat2, y_test).sum()*100/y_test.size()[0]} %")

accuracy 72.0 %


In [48]:
import gradio as gr
import numpy as np

In [49]:
def sepia (input_img):
    predict = torch.round(model(torch.tensor(input_img.reshape(-1, 64 * 64 * 3),dtype=torch.float32)/ 255.)[0,0])
    print(torch.tensor(input_img.reshape(-1, 64 * 64 * 3),dtype=torch.float64))
    if predict==1:
        return input_img,"Its a cat"
    else:
        return input_img,"Its not a cat"
    # sepia_img = input_img.dot(sepia_filter.T)
    # sepia_img /= sepia_img.max()
    # return sepia_img

demo = gr.Interface(sepia,inputs=gr.Image(shape=(64, 64)), outputs=[gr.Image(shape=(64, 64)),"text"])
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


tensor([[255., 255., 255.,  ..., 159., 183., 177.]], dtype=torch.float64)
tensor([[232., 200., 153.,  ..., 232., 199., 125.]], dtype=torch.float64)
tensor([[255., 255., 255.,  ..., 255., 255., 255.]], dtype=torch.float64)
tensor([[255., 255., 255.,  ..., 255., 255., 255.]], dtype=torch.float64)
